first load the required libraries

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [13]:
data_frame = pd.read_csv("ipynb/redoing/livestock_sample_data copy 3.csv", sep='\t')

In [14]:
data_frame.isnull().sum()

DATE         0
TOTAL        0
NSW_TOTAL    0
VIC_TOTAL    0
QLD_TOTAL    0
SA_TOTAL     0
WA_TOTAL     0
TAS_TOTAL    0
NT_TOTAL     8
dtype: int64

dropping NT_TOTAL because it has large amounts of empty data in the collumns

In [15]:
data_frame = data_frame.dropna(subset=['NT_TOTAL'])

In [16]:
data_frame['TOTAL'] = data_frame['TOTAL'].fillna(data_frame['TOTAL'].mean())

In [17]:
# Check for duplicates and remove them
print(data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()
print(data_frame.duplicated().sum())

0
0


In [18]:
# Ensure the 'DATE' column is in datetime format
data_frame['DATE'] = pd.to_datetime(data_frame['DATE'], format='%b-%Y')

In [19]:
data_frame = data_frame.drop_duplicates(subset=['DATE'])

In [20]:
# Calculate the IQR for the 'TOTAL' column to detect outliers
print(data_frame['TOTAL'].describe())
Q1 = data_frame['TOTAL'].quantile(0.25)
Q3 = data_frame['TOTAL'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are values above {Q3 + 1.5 * IQR} or below {Q1 - 1.5 * IQR}')

count     186.000000
mean      996.353226
std       104.731772
min       738.000000
25%       940.125000
50%       990.200000
75%      1061.525000
max      1345.500000
Name: TOTAL, dtype: float64
Outliers are values above 1243.6250000000002 or below 758.0249999999999


In [21]:
# Remove outliers based on IQR
data_frame = data_frame[(data_frame['TOTAL'] >= Q1 - 1.5 * IQR) & (data_frame['TOTAL'] <= Q3 + 1.5 * IQR)]
print(data_frame['TOTAL'].describe())

count     181.000000
mean      993.934807
std        94.450578
min       767.500000
25%       941.400000
50%       988.600000
75%      1056.400000
max      1212.700000
Name: TOTAL, dtype: float64


In [22]:
# Scale the 'TOTAL' column (you can choose any other numeric column if needed)
scaler = MinMaxScaler()
data_frame['TOTAL'] = scaler.fit_transform(data_frame[['TOTAL']])

In [23]:
print(data_frame.describe())

                                DATE       TOTAL   NSW_TOTAL   VIC_TOTAL  \
count                            181  181.000000  181.000000  181.000000   
mean   2002-05-12 01:03:38.784530432    0.508614  226.008287  169.941436   
min              1979-06-01 00:00:00    0.000000  157.600000  104.500000   
25%              1991-03-01 00:00:00    0.390611  201.600000  143.200000   
50%              2002-06-01 00:00:00    0.496631  226.100000  161.400000   
75%              2013-09-01 00:00:00    0.648922  248.600000  194.800000   
max              2024-12-01 00:00:00    1.000000  326.500000  270.700000   
std                              NaN    0.212153   33.135664   36.755436   

        QLD_TOTAL    SA_TOTAL    WA_TOTAL   TAS_TOTAL    NT_TOTAL  
count  181.000000  181.000000  181.000000  181.000000  181.000000  
mean   461.151934   49.388950   55.863536   23.300000    6.339227  
min    158.000000   17.100000   25.300000   12.700000    0.000000  
25%    399.200000   38.100000   44.100000  

In [24]:
data_frame.to_csv('ipynb/redoing/cleaned_bulls_data.csv', index=False)